In [213]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json
import matplotlib.pyplot as plt
from datetime import datetime
import re
from datetime import timedelta
import datetime
import time

In [224]:
max_days_to_look = 0
l_competitions = [("Premier Leauge", "https://www.skysports.com/premier-league-fixtures"),
                  ("Championship", "https://www.skysports.com/championship-fixtures"),
                  ("Scotish League", "https://www.skysports.com/scottish-football/fixtures")]

for competition,url_fixture in l_competitions:
    text_fixtures = BeautifulSoup(requests.get(url_fixture).text, "lxml")

    for fixture in text_fixtures.findAll('div', {'class': 'fixres__item'}):
        time.sleep(.5)
        for fix in BeautifulSoup(str(fixture)).findAll('a', href=True):
            if 'skysports' in fix['href']:
                l_link = fix['href'].rsplit('/',1)
                url_results = f"{l_link[0]}/stats/{l_link[1]}"

                text_results = BeautifulSoup(requests.get(url_results).text, "lxml")

                dt_data = text_results.find('div', {'class': 'sdc-site-match-header__detail'})
                dt = BeautifulSoup(str(dt_data)).find('time').next_element.replace(', ', ' ').split()
                date = " ".join(dt[2:])[:-1]
                date = date.replace('st', '').replace('nd', '').replace('rd', '').replace('th', '')

                dt_game = datetime.datetime.strptime(date, "%d %B %Y")

                days_till_feature = (dt_game - datetime.datetime.now()).days + 1

                if days_till_feature > max_days_to_look:
                    break
                #print(f"Days until feature: {days_till_feature}")

                print(f"{' '.join(dt[:2])} {date} - {competition}")

                d_stats = {}
                for team in url_results.split('/')[-3].split('-vs-'):
                    d_stats[team] = list()

                for result_data in text_results.findAll('a', {'class': 'sdc-site-last-games__result'}):
                    #print(result_data)
                    time.sleep(0.5)
                    for parts in result_data['href'].split('/'):
                        if 'vs' in parts:
                            teams = parts.split('-vs-')

                    outcome = result_data['data-outcome']
                    result = result_data.find('div', {'class': 'sdc-site-last-games__score'}).text.strip()
                    opponent = re.sub(r"\([^()]*\)", "", result_data.find('span', {'class': 'sdc-site-last-games__opponent'}).text.strip()).strip()
                    homeaway = result_data.find('span', {'class': 'sdc-site-last-games__location'}).text.strip()
                    opponent += " " + homeaway

                    d_stats[teams[0 if homeaway == '(h)' else 1]].append((outcome,result,opponent))

                #print(d_stats)
                pt = PrettyTable()
                pt.field_names = ["Team", "More Recent", "", "    ", "   ", " ", "Older"]

                for team in d_stats:
                    l_outcomes = [""]
                    l_results = [team.replace('-', ' ').title()]
                    l_opponents = [""]
                    for out in d_stats[team]:
                        l_outcomes.append(out[0])
                        l_results.append(out[1])
                        l_opponents.append(out[2])
                    pt.add_row(l_outcomes)
                    pt.add_row(l_results)
                    pt.add_row(l_opponents)
                    pt.add_row(['', '', '', '', '', '', ''])

                print(pt)
                print("\n")

        if days_till_feature > max_days_to_look:
            break

12:30pm Saturday 3 April 2021 - Premier Leauge
+----------------------+---------------+---------------+-------------+--------------+---------------+-------------+
|         Team         |  More Recent  |               |             |              |               |    Older    |
+----------------------+---------------+---------------+-------------+--------------+---------------+-------------+
|                      |      win      |      win      |     draw    |     win      |      win      |     draw    |
|       Chelsea        |     2 - 0     |     2 - 0     |    0 - 0    |    2 - 0     |     1 - 0     |    0 - 0    |
|                      | Sheff Utd (h) |  A Madrid (h) |  Leeds (a)  | Everton (h)  | Liverpool (a) | Man Utd (h) |
|                      |               |               |             |              |               |             |
|                      |      lose     |      draw     |     lose    |     win      |      draw     |     draw    |
| West Bromwich Albion | 

3:00pm Saturday 3 April 2021 - Scotish League
+------------+---------------+----------------+--------------+------------+---------------+-------------+
|    Team    |  More Recent  |                |              |            |               |    Older    |
+------------+---------------+----------------+--------------+------------+---------------+-------------+
|            |      draw     |      win       |     draw     |    win     |      lose     |     draw    |
| Ayr United |     2 - 2     |     4 - 0      |    1 - 1     |   3 - 1    |     0 - 2     |    0 - 0    |
|            |   Alloa (a)   | Elgin City (a) |  Raith (h)   | Dundee (a) |   Hearts (a)  |  Raith (a)  |
|            |               |                |              |            |               |             |
|            |      lose     |      win       |     draw     |    win     |      lose     |     lose    |
|   Clyde    |     0 - 1     |     2 - 0      |    2 - 2     |   2 - 0    |     1 - 3     |    2 - 4    |


3:00pm Saturday 3 April 2021 - Scotish League
+-------------------+----------------+-------------------------+-------------------+--------------------+--------------------+--------------------+
|        Team       |  More Recent   |                         |                   |                    |                    |       Older        |
+-------------------+----------------+-------------------------+-------------------+--------------------+--------------------+--------------------+
|                   |      draw      |           win           |        lose       |        win         |        draw        |        win         |
| Formartine United |     1 - 1      |          2 - 1          |       0 - 4       |       2 - 1        |       2 - 2        |       5 - 1        |
|                   |   Annan (h)    | Haddington Athletic (a) | East Kilbride (h) | Gala Fairydean (a) | Gala Fairydean (h) | Turriff United (a) |
|                   |                |                         |  

3:00pm Saturday 3 April 2021 - Scotish League
+----------------+----------------+-------------+----------------+---------------+----------------+----------------+
|      Team      |  More Recent   |             |                |               |                |     Older      |
+----------------+----------------+-------------+----------------+---------------+----------------+----------------+
|                |      lose      |     lose    |      draw      |      lose     |      lose      |      win       |
|    Arbroath    |     0 - 1      |    1 - 2    |     0 - 0      |     0 - 2     |     2 - 4      |     1 - 0      |
|                | Inverness (a)  | Falkirk (h) |   Hearts (h)   |   Dundee (a)  | Q of South (h) | Ayr United (a) |
|                |                |             |                |               |                |                |
|                |      draw      |     lose    |      win       |      draw     |      lose      |      draw      |
| Alloa Athletic |

3:00pm Saturday 3 April 2021 - Scotish League
+-----------------+-------------+---------------+----------------+---------------------+--------------+--------------+
|       Team      | More Recent |               |                |                     |              |    Older     |
+-----------------+-------------+---------------+----------------+---------------------+--------------+--------------+
|                 |     lose    |      win      |      lose      |         win         |     lose     |     draw     |
|    Peterhead    |    0 - 2    |     1 - 0     |     0 - 1      |        1 - 0        |    0 - 1     |    1 - 1     |
|                 |  Clyde (h)  | Dumbarton (a) | Stenh'muir (h) |     Airdrie (h)     |   Cove (a)   | Montrose (h) |
|                 |             |               |                |                     |              |              |
|                 |     draw    |      lose     |      win       |         win         |     draw     |     lose     |
| 

3:00pm Saturday 3 April 2021 - Scotish League
+---------------+--------------+---------------+----------------+---------------+-----------------------------+----------------+
|      Team     | More Recent  |               |                |               |                             |     Older      |
+---------------+--------------+---------------+----------------+---------------+-----------------------------+----------------+
|               |     win      |      draw     |      lose      |      lose     |             win             |      win       |
|   Elgin City  |    2 - 1     |     1 - 1     |     0 - 4      |     1 - 2     |            4 - 0            |     2 - 0      |
|               | Brechin (a)  |  Stirling (h) | Ayr United (h) | Edinburgh (h) | Civil Service Strollers (h) | Stenh'muir (h) |
|               |              |               |                |               |                             |                |
|               |     lose     |      lose     |   

In [209]:
pt = PrettyTable()
pt.field_names = ["Team", "More Recent", "", "    ", "   ", " ", "Older"]

for team in d_stats:
    l_outcomes = [""]
    l_results = [team.replace('-', ' ').title()]
    l_opponents = [""]
    for out in d_stats[team]:
        l_outcomes.append(out[0])
        l_results.append(out[1])
        l_opponents.append(out[2])
    pt.add_row(l_outcomes)
    pt.add_row(l_results)
    pt.add_row(l_opponents)

print(pt)

+----------------------+---------------+---------------+-------------+--------------+---------------+-------------+
|         Team         |  More Recent  |               |             |              |               |    Older    |
+----------------------+---------------+---------------+-------------+--------------+---------------+-------------+
|                      |      win      |      win      |     draw    |     win      |      win      |     draw    |
|       Chelsea        |     2 - 0     |     2 - 0     |    0 - 0    |    2 - 0     |     1 - 0     |    0 - 0    |
|                      | Sheff Utd (h) |  A Madrid (h) |  Leeds (a)  | Everton (h)  | Liverpool (a) | Man Utd (h) |
|                      |      lose     |      draw     |     lose    |     win      |      draw     |     draw    |
| West Bromwich Albion |     0 - 1     |     0 - 0     |    0 - 1    |    1 - 0     |     0 - 0     |    1 - 1    |
|                      |  C Palace (a) | Newcastle (h) | Everton (h) | B

In [196]:
d_stats[team]

[('lose', '0 - 1', 'C Palace (a)'),
 ('draw', '0 - 0', 'Newcastle (h)'),
 ('lose', '0 - 1', 'Everton (h)'),
 ('win', '1 - 0', 'Brighton (h)'),
 ('draw', '0 - 0', 'Burnley (a)'),
 ('draw', '1 - 1', 'Man Utd (h)')]

In [191]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Teams", "Area", "Population", "Annual Rainfall"]

x.add_row(["Adelaide", 1295, 1158259, 600.5])
x.add_row(["Brisbane", 5905, 1857594, 1146.4])
x.add_row(["Darwin", 112, 120900, 1714.7])
x.add_row(["--------", "--------", "--------", "--------"])
x.add_row(["Hobart", 1357, 205556, 619.5])
x.add_row(["Sydney", 2058, 4336374, 1214.8])
x.add_row(["Melbourne", 1566, 3806092, 646.9])
x.add_row(["Perth", 5386, 1554769, 869.4])

print(x)

+-----------+----------+------------+-----------------+
| City name |   Area   | Population | Annual Rainfall |
+-----------+----------+------------+-----------------+
|  Adelaide |   1295   |  1158259   |      600.5      |
|  Brisbane |   5905   |  1857594   |      1146.4     |
|   Darwin  |   112    |   120900   |      1714.7     |
|  -------- | -------- |  --------  |     --------    |
|   Hobart  |   1357   |   205556   |      619.5      |
|   Sydney  |   2058   |  4336374   |      1214.8     |
| Melbourne |   1566   |  3806092   |      646.9      |
|   Perth   |   5386   |  1554769   |      869.4      |
+-----------+----------+------------+-----------------+


In [117]:
for fixture in text_fixtures.findAll('div', {'class': 'fixres__item'}):
    print(fixture)
    break
    for fix in BeautifulSoup(str(fixture)).findAll('a', href=True):
        if 'skysports' in fix['href']:
            l_link = fix['href'].rsplit('/',1)
            ref = f"{l_link[0]}/stats/{l_link[1]}"
            print(ref)
            print("---")

<div class="fixres__item">
<a class="matches__item matches__link" data-high-coverage="true" data-item-id="429129" data-status="KO" href="https://www.skysports.com/football/chelsea-vs-west-bromwich-albion/429129">
<span class="matches__item-col matches__label"></span>
<span class="matches__item-col matches__participant matches__participant--side1">
<span class="swap-text--bp30" title="Chelsea">
<span class="swap-text__target">Chelsea</span>
</span>
</span>
<span class="matches__item-col matches__status">
<span class="matches__teamscores">
<span class="matches__teamscores-side">
                0            </span>
<span class="matches__teamscores-side">
                0            </span>
</span>
<span class="matches__date">
        12:30    </span>
</span>
<span class="matches__item-col matches__participant matches__participant--side2">
<span class="swap-text--bp30" title="W Brom">
<span class="swap-text__target">West Bromwich Albion</span>
</span>
</span>
<span class="matches__item-c

In [150]:
dt_data = text_results.find('div', {'class': 'sdc-site-match-header__detail'})

dt = BeautifulSoup(str(dt_data)).find('time').next_element.replace(', ', ' ').split()
print(" ".join(dt[:2]))
print(" ".join(dt[2:]))

12:30pm Saturday
3rd April 2021.


In [116]:
url_results = "https://www.skysports.com/football/chelsea-vs-west-bromwich-albion/stats/429129"
text_results = BeautifulSoup(requests.get(url_results).text, "lxml")

d_stats = {}
for team in url_results.split('/')[-3].split('-vs-'):
    d_stats[team] = list()

for result_data in text_results.findAll('a', {'class': 'sdc-site-last-games__result'}):
    #print(result_data)
    for parts in result_data['href'].split('/'):
        if 'vs' in parts:
            teams = parts.split('-vs-')
            
    outcome = result_data['data-outcome']
    result = result_data.find('div', {'class': 'sdc-site-last-games__score'}).text.strip()
    opponent = re.sub(r"\([^()]*\)", "", result_data.find('span', {'class': 'sdc-site-last-games__opponent'}).text.strip()).strip()
    homeaway = result_data.find('span', {'class': 'sdc-site-last-games__location'}).text.strip()
    opponent += " " + homeaway
    
    d_stats[teams[0 if homeaway == '(h)' else 1]].append((outcome,result,opponent))

d_stats

{'chelsea': [('win', '2 - 0', 'Sheff Utd (h)'),
  ('win', '2 - 0', 'A Madrid (h)'),
  ('draw', '0 - 0', 'Leeds (a)'),
  ('win', '2 - 0', 'Everton (h)'),
  ('win', '1 - 0', 'Liverpool (a)'),
  ('draw', '0 - 0', 'Man Utd (h)')],
 'west-bromwich-albion': [('lose', '0 - 1', 'C Palace (a)'),
  ('draw', '0 - 0', 'Newcastle (h)'),
  ('lose', '0 - 1', 'Everton (h)'),
  ('win', '1 - 0', 'Brighton (h)'),
  ('draw', '0 - 0', 'Burnley (a)'),
  ('draw', '1 - 1', 'Man Utd (h)')]}

In [156]:
month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [157]:
len(month)

12

In [166]:

date = "11th April 2021"
date = date.replace('st', '').replace('nd', '').replace('rd', '').replace('th', '')

dt_game = datetime.datetime.strptime(date, "%d %B %Y")

days_till_feature = (dt_game - datetime.datetime.now()).days

In [225]:
for idx,val in enumerate(["abc", "def"]):
    print(idx)

0
1


In [227]:
def team_moment(team, score):
    if score > 14:
        moment = "wonderful"
    elif score > 7:
        moment = "really good"
    elif score > 0:
        moment = "good"
    elif score > -7:
        moment = "bad"
    elif score > -14:
        moment = "really bad"
    else:
        moment = "disgusting"
    
    print(f"{team} is in a {moment} moment")

In [231]:
def bet_confidence(score):
    if score > 35:
        print("This bet is GIVING AWAY MONEY FOR FREE")
    if score > 28:
        print("This bet is EASY MONEY")
    elif score > 21:
        print("This bet is a GREAT")
    elif score > 14:
        print("This bet is a GOOD")
    elif score > 7:
        print("This bet is OKAY")
    else:
        print("This bet is EXTREMELY RISKY")

In [234]:
for val,val1 in zip([1,2,3], ["1","5","3"]):
    print(val1)

1
5
3
